In [1]:
#run in the script folder
ls ../Data
#samtools sort pb_467_2_sr_blasr.bam > pb_sorted.bam
#samtools index pb_sorted.bam

hg19_chr17.fasta		pb_467_2_sr_blasr.bam
hg19_chr17.fasta.fai		pb_sorted.bam
onlyTwoSNP.vcf			pb_sorted.bam.bai
pb_467_1_minor_variants.vcf


In [ ]:
hehe-3:PacBioRead apple$ samtools view /Users/apple/Documents/AppliedBioinformatics/Data/pb_467_2_sr_blasr.bam  | wc -l
   88760

In [3]:
#samtools view -h ../Data/pb_sorted.bam 'chr17:7578263-7578263' 'chr17:7579312-7579312' \
#| wc -l
#filter only the chr17 reads left
samtools view -h -b  ../Data/pb_sorted.bam 'chr17'> ../Data/chr17.bam
#filter only reads mapped through the two position left
#region.bed:chr17	7578262	7578263; region2.bed:chr17	7579311	7579312
#Was trying to do in samtools, but output from first position selection to be indexed.
#TODO: generation of bed file(also bed files for generating realignment sequences)
bedtools intersect -abam ../Data/chr17.bam -b ../Data/region.bed -wa \
|bedtools intersect -abam stdin -b ../Data/region2.bed -wa |samtools view - | wc -l
#create longer selection region
#region.bed:chr17	7578212, 7578313; region2.bed:chr17	7579261, 7579362 chr 177578212 7578313
bedtools intersect -abam ../Data/chr17.bam -b ../Data/region50P1.bed -wa \
|bedtools intersect -abam stdin -b ../Data/region50P2.bed -wa |samtools view - | wc -l
#- Please ensure that your file is TAB delimited (e.g., cat -t FILE)
bedtools intersect -abam ../Data/chr17.bam -b ../Data/region50P1.bed  -wa \
| bedtools intersect -abam stdin -b ../Data/region50P1_1.bed  -wa \
| bedtools intersect -abam stdin -b ../Data/region50P2.bed  -wa \
| bedtools intersect -abam stdin -b ../Data/region50P2_1.bed  -wa | samtools view - | wc -l

      81


In [ ]:
#save the selected reads(PosA&PosB) to bam file
bedtools intersect -abam ../Data/chr17.bam -b ../Data/region.bed -wa \
|bedtools intersect -abam stdin -b ../Data/region2.bed -wa > ../Data/selected.bam
#TODO: generate fastQ and do factQC
#TODO: check if the selected reads mapped through the two position. (samtools tview need sort index)
#TODO: function that generates bed file.

bedtools intersect -abam ../Data/chr17.bam -b ../Data/region50P1.bed  -wa \
| bedtools intersect -abam stdin -b ../Data/region50P1_1.bed  -wa \
| bedtools intersect -abam stdin -b ../Data/region50P2.bed  -wa \
| bedtools intersect -abam stdin -b ../Data/region50P2_1.bed  -wa > ../Data/120bpSelected.bam
#TODO delect the bed file

In [3]:
#BAM to fastQ using samtools
samtools bam2fq ../Data/selected.bam > ../Data/selected.fastq

[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 81 reads


In [4]:
#BAM to fastQ using bedtools
bedtools bamtofastq -i ../Data/selected.bam -fq ../Data/bedtools.fastq

In [5]:
#base quality not stored column 11=* => no quality info in fastQ
samtools view ../Data/selected_sorted.bam | awk '{print $1,  $11'} | head -5

m170922_223647_42237_c101229972550000001823291512091717_s1_p0/105503/20133_30014/0_9881 *
m170922_223647_42237_c101229972550000001823291512091717_s1_p0/105503/40045_50088/0_10043 *
m170922_223647_42237_c101229972550000001823291512091717_s1_p0/105503/30061_39995/0_9934 *
m170922_223647_42237_c101229972550000001823291512091717_s1_p0/105503/0_10121/0_10121 *
m170922_223647_42237_c101229972550000001823291512091717_s1_p0/105503/10167_20088/0_9921 *


In [6]:
#fastq to fasta
#cat bedtools.fastq  | paste - - - - | awk -F '\t' '{print ">"$1 ; print $2;}'> bedtoolsSeq.fa
cat ../Data/bedtools.fastq  | paste - - - - | awk -F '\t' '{print ">"$1 ; print $2;}'| grep -c ">"

81


In [9]:
#generating sequences(referential)
#TODO automated generation of Ref and Alt sequences
#TODO 1/0 base when selecting the seq and index the Position of interest
#increase the length of generated seq
#
bedtools getfasta -fi ../Data/hg19_chr17_changeName.fasta -bed ../Data/bed_selectSeq.bed \
-fo ../Data/test_realign.fa

In [10]:
samtools view selected.bam | less -S

bash: blastn: command not found


: 127

In [ ]:
  501  export LC_ALL=en_US.UTF-8
  502  export LANG=en_US.UTF-8

#using UPPMAX
#sent file to the server
rsync -P ./hg19_chr17_changeName.fasta  zhiwei94@rackham.UPPMAX.uu.se:/home/zhiwei94/AppliedBio/Applied_Bioinformatics_Xdrop/hg19

#load software
module load bioinfo-tools
module spider blasr/5.3-20171117-f72428d

[zhiwei94@rackham2 Bash]$ blasr ../Query/query_15bpUp_15bpDown.fasta tmp.fasta -minMatch 5 -sam -hitPolicy randombest -out tmpResult.sam
[zhiwei94@rackham2 Bash]$ blasr ../Query/query_15bpUp_15bpDown.fasta ../hg19/hg19_chr17_changeName.fasta -sam -out tmpRestul.sam
#map bam to fastq
blasr 120bpSelected.bam  hg19_chr17_changeName.fasta --bam --out tmpResult.bam

[zhiwei94@rackham2 Bash]$ blasr ../PacBioRead/120bpSelected.bam  ../hg19/hg19_chr17_changeName.fasta  --bam --out tmpResult.bam
[INFO] 2018-11-20T20:50:51 [blasr] started.
terminate called after throwing an instance of 'std::runtime_error'
  what():  malformed record name
Aborted (core dumped)

[zhiwei94@rackham2 Bash]$ blasr tmp.fasta  ../hg19/hg19_chr17_changeName.fasta  --bam --out tmpResult.bam
[INFO] 2018-11-20T21:21:29 [blasr] started.
ERROR, can not convert non-pacbio reads to pbbam record.

blasr tmp.fasta  ../hg19/hg19_chr17_changeName.fasta
https://github.com/PacificBiosciences/blasr/wiki/Advanced-options:-to-align-very-short-reads 
blasr ../Query/query_25bpUP_25bpDown.fasta tmp.fasta --minMatch 8 --minSubreadLength 1 --minReadLength 1 --maxScore 200
# add -m 0 human readable output
[zhiwei94@rackham2 Bash]$ blasr ../Query/query_25bpUP_25bpDown.fasta tmp.fasta --minMatch 8 --minSubreadLength 1 --minReadLength 1 --maxScore 400 -m 0